In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split,  StratifiedKFold, KFold,GridSearchCV, cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import  PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import datasets, ensemble
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
# import xgboost import XGBRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
import warnings 
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/playground-series-s3e15/sample_submission.csv
/kaggle/input/playground-series-s3e15/data.csv


### Import the data

In [2]:
df = pd.read_csv('/kaggle/input/playground-series-s3e15/data.csv')          
df.head()

,id,author,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,0,Thompson,tube,7.00,3770.0,0.1754,NaN,10.8,432.0,3.6
1,1,Thompson,tube,NaN,6049.0,-0.0416,10.3,10.3,762.0,6.2
2,2,Thompson,NaN,13.79,2034.0,0.0335,7.7,7.7,457.0,2.5
3,3,Beus,annulus,13.79,3679.0,-0.0279,5.6,15.2,2134.0,3.0
4,4,NaN,tube,13.79,686.0,NaN,11.1,11.1,457.0,2.8


In [3]:
len(df)

31644

### Check the null values

In [4]:
df.isnull().sum()

id                         0
author                  5024
geometry                5500
pressure [MPa]          4452
mass_flux [kg/m2-s]     4791
x_e_out [-]            10415
D_e [mm]                5488
D_h [mm]                4589
length [mm]             4759
chf_exp [MW/m2]            0
dtype: int64

In [5]:
df.head()

,id,author,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,0,Thompson,tube,7.00,3770.0,0.1754,NaN,10.8,432.0,3.6
1,1,Thompson,tube,NaN,6049.0,-0.0416,10.3,10.3,762.0,6.2
2,2,Thompson,NaN,13.79,2034.0,0.0335,7.7,7.7,457.0,2.5
3,3,Beus,annulus,13.79,3679.0,-0.0279,5.6,15.2,2134.0,3.0
4,4,NaN,tube,13.79,686.0,NaN,11.1,11.1,457.0,2.8


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31644 entries, 0 to 31643
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   31644 non-null  int64  
 1   author               26620 non-null  object 
 2   geometry             26144 non-null  object 
 3   pressure [MPa]       27192 non-null  float64
 4   mass_flux [kg/m2-s]  26853 non-null  float64
 5   x_e_out [-]          21229 non-null  float64
 6   D_e [mm]             26156 non-null  float64
 7   D_h [mm]             27055 non-null  float64
 8   length [mm]          26885 non-null  float64
 9   chf_exp [MW/m2]      31644 non-null  float64
dtypes: float64(7), int64(1), object(2)
memory usage: 2.4+ MB


In [7]:
df.describe()

,id,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
count,31644.000000,27192.000000,26853.000000,21229.000000,26156.000000,27055.000000,26885.000000,31644.000000
mean,15821.500000,10.640747,3068.011023,-0.000453,8.629255,14.174330,832.987391,3.796985
std,9134.980296,4.333683,1777.032080,0.100911,5.185692,19.838489,672.299239,1.983991
min,0.000000,0.100000,0.000000,-0.866700,1.000000,1.000000,10.000000,0.800000
25%,7910.750000,6.890000,1519.000000,-0.046600,5.000000,5.600000,318.000000,2.400000
50%,15821.500000,11.070000,2731.000000,0.003800,7.800000,10.000000,610.000000,3.400000
75%,23732.250000,13.790000,4069.000000,0.064800,10.800000,11.500000,914.000000,4.600000
max,31643.000000,20.680000,7975.000000,0.232000,37.500000,120.000000,3048.000000,19.300000


In [8]:
df.duplicated().sum()

0

In [9]:
df.nunique()

id                     31644
author                    10
geometry                   3
pressure [MPa]           144
mass_flux [kg/m2-s]      733
x_e_out [-]             1682
D_e [mm]                  43
D_h [mm]                  49
length [mm]               70
chf_exp [MW/m2]          109
dtype: int64

In [10]:
### Rename the columns

df = df.rename(columns={'pressure [MPa]': 'pressure', 
                 'mass_flux [kg/m2-s]': 'mass_flux',
                 'x_e_out [-]': 'x_e_out',
                 'D_e [mm]': 'D_e',
                 'D_h [mm]': 'D_h',
                 'length [mm]': 'length',
                 'chf_exp [MW/m2]': 'chf_exp'})

In [11]:
df.columns

Index(['id', 'author', 'geometry', 'pressure', 'mass_flux', 'x_e_out', 'D_e',
       'D_h', 'length', 'chf_exp'],
      dtype='object')

### Feature Imputation

## Simple Imputation

In [12]:
simple_imputer_cat = SimpleImputer(strategy='most_frequent')
simple_imputer_num = SimpleImputer(strategy='mean')

In [13]:
def imputation(data):
    df_imp = data.copy()
    for i in data.columns:
        if data[i].dtypes == 'object':
            df_imp[i] = simple_imputer_cat.fit_transform(df_imp[[i]])
        else:
            df_imp[i] = simple_imputer_num.fit_transform(df_imp[[i]])
            
    return df_imp

In [14]:
### lets drop the target feature
df_train = df.drop(columns='x_e_out', axis=1)
df_imp = imputation(df_train)

In [15]:
df_imp.isnull().sum()

id           0
author       0
geometry     0
pressure     0
mass_flux    0
D_e          0
D_h          0
length       0
chf_exp      0
dtype: int64

In [16]:
### get the unique values features. it seems that only 2 features has less categorical values

unqiue_features = [df[i].unique() for i in df.columns if df[i].dtypes == 'object']
print(unqiue_features)

[array(['Thompson', 'Beus', nan, 'Peskov', 'Janssen', 'Weatherhead',
       'Inasaka', 'Williams', 'Mortimore', 'Richenderfer', 'Kossolapov'],
      dtype=object), array(['tube', nan, 'annulus', 'plate'], dtype=object)]


In [17]:
# imputer = IterativeImputer(random_state=100, max_iter=10)
# imputer.fit(df.iloc[:, :-1])

In [18]:
df_imp.head()

,id,author,geometry,pressure,mass_flux,D_e,D_h,length,chf_exp
0,0.0,Thompson,tube,7.000000,3770.0,8.629255,10.8,432.0,3.6
1,1.0,Thompson,tube,10.640747,6049.0,10.300000,10.3,762.0,6.2
2,2.0,Thompson,tube,13.790000,2034.0,7.700000,7.7,457.0,2.5
3,3.0,Beus,annulus,13.790000,3679.0,5.600000,15.2,2134.0,3.0
4,4.0,Thompson,tube,13.790000,686.0,11.100000,11.1,457.0,2.8


In [19]:


df_imp = df_imp.drop('id',axis=1)
df_final  = df_imp.copy()
df_final['x_e_out'] = df['x_e_out']
df_final['x_e_out'].isnull()
df_final = pd.get_dummies(df_final, columns=['author', 'geometry'])
df_train_data = df_final[~df_final['x_e_out'].isna()]
df_test_data = df_final[df_final['x_e_out'].isna()]
df_train_data = df_train_data.reset_index()
df_train_data = df_train_data.drop('index', axis=1)

In [20]:
df_test_data = df_test_data.drop(columns='x_e_out', axis=1)
df_test_data = df_test_data.reset_index()
df_test_data = df_test_data.drop('index', axis=1)
df_test_data.head()

,pressure,mass_flux,D_e,D_h,length,chf_exp,author_Beus,author_Inasaka,author_Janssen,author_Kossolapov,author_Mortimore,author_Peskov,author_Richenderfer,author_Thompson,author_Weatherhead,author_Williams,geometry_annulus,geometry_plate,geometry_tube
0,13.790000,686.000000,11.100000,11.1,457.0,2.8,0,0,0,0,0,0,0,1,0,0,0,0,1
1,18.000000,750.000000,10.000000,10.0,1650.0,2.2,0,0,0,0,0,1,0,0,0,0,0,0,1
2,10.640747,3068.011023,1.900000,1.9,152.0,3.2,0,0,0,0,0,0,0,1,0,0,0,0,1
3,6.890000,7500.000000,8.629255,12.8,1930.0,4.8,0,0,0,0,0,0,0,1,0,0,0,0,1
4,15.510000,1355.000000,5.600000,15.2,2134.0,2.1,1,0,0,0,0,0,0,0,0,0,1,0,0


In [21]:
X = df_train_data.drop(columns = 'x_e_out',axis=1)
y = df_train_data['x_e_out']

In [22]:
params = {'colsample_bytree' : 0.8,  
          'gamma' : 0.01, 
          'learning_rate' : 0.01, 
          'max_depth' : 5,  
          'min_child_weight' : 50, 
          'n_estimators' : 1000, 
          'subsample' : 0.7}

In [23]:
kfold = KFold(n_splits=10, shuffle=True, random_state=0)
pred_xgb_simple = np.zeros(df_test_data.shape[0])
# pred_cat = np.zeros(test_df.shape[0])
# pred_lgb = np.zeros(test_df.shape[0])
rmse_xgb=[]  

rmse = []# list contains rmse for each fold
n=0

for i, (train_index, valid_index) in enumerate(kfold.split(X, y)):
    x_train, y_train = X.iloc[train_index], y.iloc[train_index]
    x_valid, y_valid = X.loc[valid_index], y.iloc[valid_index]
    
    # XGBoost regressor
    model_xgb = XGBRegressor(**params)
    model_xgb.fit(x_train,y_train,eval_set=[(x_valid,y_valid)],early_stopping_rounds=100,verbose=False) 
    pred_xgb_simple+=model_xgb.predict(df_test_data)/kfold.n_splits
    rmse_xgb.append(mean_squared_error(y_valid, model_xgb.predict(x_valid), squared=False))
    
#     # CatBoost regressor
#     model_cat = cat.CatBoostRegressor(**cat_params)
#     model_cat.fit(x_train,y_train,eval_set=[(x_valid,y_valid)],early_stopping_rounds=100,verbose=False) 
#     pred_cat+=model_cat.predict(test_df)/kfold.n_splits
#     rmse_cat.append(mean_squared_error(y_valid, model_cat.predict(x_valid), squared=False))
    
    
#     # LightGBM regressor
#     model_lgb = LGBMRegressor(**study_lgbm.best_params)
#     model_lgb.fit(x_train,y_train,eval_set=[(x_valid,y_valid)],early_stopping_rounds=100,verbose=False) 
#     pred_lgb+=model_lgb.predict(test_df)/kfold.n_splits
#     rmse_lgb.append(mean_squared_error(y_valid, model_lgb.predict(x_valid), squared=False))
    
#     rmse_val = (rmse_xgb[n]+ rmse_cat[n]+ rmse_lgb[n]) / 3    
#     rmse.append(rmse_val)
    print(f"fold: {n+1} rmse xgb: {rmse_xgb[n]}")
    n+=1

print(np.mean(rmse_xgb))

fold: 1 rmse xgb: 0.07674336173719133
fold: 2 rmse xgb: 0.07571137618673736
fold: 3 rmse xgb: 0.07318831303056739
fold: 4 rmse xgb: 0.07574160808590204
fold: 5 rmse xgb: 0.0749294518485237
fold: 6 rmse xgb: 0.0731792787930007
fold: 7 rmse xgb: 0.0731992093565595
fold: 8 rmse xgb: 0.07306174171127816
fold: 9 rmse xgb: 0.07279341415689007
fold: 10 rmse xgb: 0.07378969007873658
0.07423374449853869


#### MICE Imputation

In [24]:
df_train = df_train.drop('id', axis=1)

In [25]:
### encode the categoracal variables
label_encoder = preprocessing.LabelEncoder()
df_train['author'] = label_encoder.fit_transform(df['author'])
df_train['geometry'] = label_encoder.fit_transform(df['geometry'])


In [26]:
### create the imputer instance

imputer = IterativeImputer(random_state=100, max_iter=10)

In [27]:
imputer.fit(df_train)
df_mice = imputer.transform(df_train)

In [28]:
df_mice

array([[   7.        ,    2.        ,    7.        , ...,   10.8       ,
         432.        ,    3.6       ],
       [   7.        ,    2.        ,    7.87861191, ...,   10.3       ,
         762.        ,    6.2       ],
       [   7.        ,    3.        ,   13.79      , ...,    7.7       ,
         457.        ,    2.5       ],
       ...,
       [   7.        ,    3.        ,   18.27      , ...,    3.        ,
         150.        ,    2.3       ],
       [   7.        ,    2.        ,    6.89      , ...,   23.6       ,
        1972.        ,    3.7       ],
       [  10.        ,    2.        ,    6.89      , ...,   12.8       ,
        1930.        ,    3.3       ]])

In [29]:
df_train.loc[:, :] = df_mice

In [30]:
df_train.isnull().sum()

author       0
geometry     0
pressure     0
mass_flux    0
D_e          0
D_h          0
length       0
chf_exp      0
dtype: int64

In [31]:

df_final_mice  = df_train.copy()
df_final_mice['x_e_out'] = df['x_e_out']
df_final_mice['x_e_out'].isnull()
df_final_mice = pd.get_dummies(df_final_mice, columns=['author', 'geometry'])
df_train_data_mice = df_final_mice[~df_final_mice['x_e_out'].isna()]
df_test_data_mice = df_final_mice[df_final_mice['x_e_out'].isna()]
df_train_data_mice = df_train_data_mice.reset_index()
df_train_data_mice = df_train_data_mice.drop('index', axis=1)

In [32]:
df_test_data_mice = df_test_data_mice.drop(columns='x_e_out', axis=1)
df_test_data_mice = df_test_data_mice.reset_index()
df_test_data_mice = df_test_data_mice.drop('index', axis=1)
df_test_data_mice.head()

,pressure,mass_flux,D_e,D_h,length,chf_exp,author_0.0,author_1.0,author_2.0,author_3.0,...,author_5.0,author_6.0,author_7.0,author_8.0,author_9.0,author_10.0,geometry_0.0,geometry_1.0,geometry_2.0,geometry_3.0
0,13.790000,686.000000,11.100000,11.1,457.0,2.8,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,18.000000,750.000000,10.000000,10.0,1650.0,2.2,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,13.689635,2829.407261,1.900000,1.9,152.0,3.2,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
3,6.890000,7500.000000,15.727711,12.8,1930.0,4.8,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,15.510000,1355.000000,5.600000,15.2,2134.0,2.1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [33]:
X = df_train_data_mice.drop(columns = 'x_e_out',axis=1)
y = df_train_data_mice['x_e_out']

In [34]:
kfold = KFold(n_splits=10, shuffle=True, random_state=0)
pred_xgb = np.zeros(df_test_data_mice.shape[0])
# pred_cat = np.zeros(test_df.shape[0])
# pred_lgb = np.zeros(test_df.shape[0])
rmse_xgb=[]  

rmse = []# list contains rmse for each fold
n=0

for i, (train_index, valid_index) in enumerate(kfold.split(X, y)):
    x_train, y_train = X.iloc[train_index], y.iloc[train_index]
    x_valid, y_valid = X.loc[valid_index], y.iloc[valid_index]
    
    # XGBoost regressor
    model_xgb = XGBRegressor(**params)
    model_xgb.fit(x_train,y_train,eval_set=[(x_valid,y_valid)],early_stopping_rounds=100,verbose=False) 
    pred_xgb+=model_xgb.predict(df_test_data_mice)/kfold.n_splits
    rmse_xgb.append(mean_squared_error(y_valid, model_xgb.predict(x_valid), squared=False))
    

    print(f"fold: {n+1} rmse xgb: {rmse_xgb[n]}")
    n+=1

print(np.mean(rmse_xgb))

fold: 1 rmse xgb: 0.07685814984671106
fold: 2 rmse xgb: 0.07603496315419228
fold: 3 rmse xgb: 0.07345786198944831
fold: 4 rmse xgb: 0.07598536638250696
fold: 5 rmse xgb: 0.0752338129683224
fold: 6 rmse xgb: 0.07347969354558967
fold: 7 rmse xgb: 0.07337202001418554
fold: 8 rmse xgb: 0.07327834234645629
fold: 9 rmse xgb: 0.07305776393036915
fold: 10 rmse xgb: 0.07383310597019205
0.07445910801479735


### Add the submission

In [35]:
submission = pd.read_csv('/kaggle/input/playground-series-s3e15/sample_submission.csv')
submission.head()

,id,x_e_out [-]
0,4,0.0
1,7,0.0
2,10,0.0
3,12,0.0
4,23,0.0


In [36]:
submission['x_e_out [-]'] = pred_xgb

In [37]:
submission.to_csv('submission.csv', index=False)